In [1]:
import spacy
import pandas as pd
import numpy as np

In [2]:
nlp_org = spacy.load("en_core_web_sm")

In [3]:
model_path = r'custom_NER_model'
# Loading the model from the directory
print("Loading from", model_path)
nlp_custom = spacy.load(model_path)
assert nlp_custom.get_pipe("ner")

Loading from custom_NER_model


C:\Users\kartik.nerwal\AppData\Roaming\Python\Python39\site-packages\spacy\util.py:865: UserWarning: [W095] Model 'en_NER_811' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [38]:
file_path = r'Text_Data.txt'
res = []
with open(file_path) as file:
    lst = file.readlines()
    for i in lst:
        while '(' and ')' in i:
                i=i.replace(i[i.index('('):i.index(')')+1],'')
        i = i.replace('&','and')    
        i = i.replace("Intersection","").replace("Int", "Intersection")   
        i = i.replace('FT','feet')
        i = i.replace('Ft','feet')
        i = i.replace('Appx','approximately')
        i = i.replace('\n','')
        res.append(i)

In [39]:
res

['Adjacent to Fowler Ave and Near Intersection  of Fowler Ave and Robinwood Ave  ',
 'Opposite to PLOT 35 feet to North ',
 'Front and back of the Property address ',
 'From Intersection  of James Road and Washington Ave Go South on James approximately 100 feet to Site on Right. LOC Entire Vacant ',
 'Adjacent to Plot 3970 50 feet North ',
 'Front of Property 100 feet West ',
 '30l N Right to Property address 50 feet to Site ',
 'Front of PLOT 200l N of Arnold Industrial Way on West Side of Port Chicago HWY ',
 'Right to Property address towards Junction 100 feet for Site ',
 'Opposite Front of Property 80 feet South ',
 'In between property and adjacent plot 35 feet West ',
 'Opposite to Pizza King 35l N Near junction on S10 street ',
 '55 feet East Adjacent to Property ',
 'Opposite to PLOT 835 West ',
 '80 feet West of the Plot ',
 'West to the plot at Intersection of N Blvd and W Anderson St ',
 'Front of PLOT 40 feet North ',
 'Opposite to Samoa Club 20l N at junction ',
 'Right t

In [40]:
def get_entities_with_org(line):
    doc = nlp_org(line)
    entity = []
    for en in doc.ents:
        entity.append(en.text)
    return entity


def get_entities_with_custom(line):
    doc = nlp_custom(line)
    entity = []
    for en in doc.ents:
        entity.append(en.text)
    return entity

def split_on_entity(en, l_seg):
    x = []
    x.append(line[:line.find(en)])
    x.append(en)
    x.append(line[line.find(en)+len(en):])
    for i in x:
        if(len(i.strip()) < 3):
            x.remove(i)
    
    return x


all_entities = []
all_lines = []
for line in res:
    line = line.strip()
    entity = get_entities_with_custom(line)
    line_segments = []
    for num, en in enumerate(entity):
        all_entities.append(en)
        x = split_on_entity(en, line.strip())
#         print(x)
        for seg in x:
            if(seg == en):
                line_segments.append(seg)
            elif(num == len(entity)-1):
                line_segments.append(seg)
            else:
                line = seg
#         break
    all_lines.append(line_segments)
#     break
    
    

In [41]:
all_lines

[['Adjacent to',
  ' Fowler Ave and ',
  'Near Intersection',
  '  of Fowler Ave and Robinwood Ave'],
 ['Opposite to', ' PLOT 35 feet to North'],
 ['Front', ' and ', 'back', ' of the Property address'],
 ['Intersection  ',
  'of James Road and Washington Ave Go ',
  'South on',
  ' James approximately 100 feet to Site on Right. LOC Entire Vacant'],
 ['Adjacent to', ' Plot 3970 50 feet North'],
 ['Front of', ' Property 100 feet West'],
 ['30l N ', 'Right to', ' Property address 50 feet to Site'],
 ['Front of',
  ' PLOT 200l N of Arnold Industrial Way on West Side of Port Chicago HWY'],
 ['Right to', ' Property address towards Junction 100 feet for Site'],
 ['Opposite', 'Front of', ' Property 80 feet South'],
 ['In between', ' property and ', 'adjacent', ' plot 35 feet West'],
 ['Opposite to', ' Pizza King 35l N ', 'Near junction', ' on S10 street'],
 ['55 feet East ', 'Adjacent to', ' Property'],
 ['Opposite to', ' PLOT 835 West'],
 ['80 feet ', 'West of', ' the Plot'],
 ['West to', ' t

In [42]:
df = pd.DataFrame(columns={"Input"})

In [43]:
df.Input = res

In [44]:
cols = {}
max_len = 0
for i in all_lines:
    if(max_len < len(i)):
        max_len = len(i)

for i in range(max_len):
    cols[i] = []

for line in all_lines:
    for j in range(len(line)):
        cols[j].append(line[j])
    for k in range(len(line), len(line)+(max_len-len(line))):
        cols[k].append(None)


In [45]:
for col in cols:
    df[col] = cols[col]

In [46]:
df

,Input,0,1,2,3
0,Adjacent to Fowler Ave and Near Intersection ...,Adjacent to,Fowler Ave and,Near Intersection,of Fowler Ave and Robinwood Ave
1,Opposite to PLOT 35 feet to North,Opposite to,PLOT 35 feet to North,None,None
2,Front and back of the Property address,Front,and,back,of the Property address
3,From Intersection of James Road and Washingto...,Intersection,of James Road and Washington Ave Go,South on,James approximately 100 feet to Site on Right...
4,Adjacent to Plot 3970 50 feet North,Adjacent to,Plot 3970 50 feet North,None,None
5,Front of Property 100 feet West,Front of,Property 100 feet West,None,None
6,30l N Right to Property address 50 feet to Site,30l N,Right to,Property address 50 feet to Site,None
7,Front of PLOT 200l N of Arnold Industrial Way ...,Front of,PLOT 200l N of Arnold Industrial Way on West ...,None,None
8,Right to Property address towards Junction 100...,Right to,Property address towards Junction 100 feet fo...,None,None
9,Opposite Front of Property 80 feet South,Opposite,Front of,Property 80 feet South,None


In [47]:
df.to_csv('Original_to_Broken_Info.csv', index=False)